### Stock Summary from Yahoo

In [3]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("No API key was found")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
openai = OpenAI()

In [7]:
from urllib.parse import urlencode
import datetime

headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

In [ ]:
class StockWebsite:
    def __init__(self, stock_symbol):
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"
    
    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params={"frequency":"1wk" , "period1":datetime_year_ago.timestamp(), "period2":datetime_now.timestamp()}
        url = self.__build_url(params)
        response=requests.get(url,headers=headers)
        soup=BeautifulSoup(response.content, "html.parser")

        title = soup.title.string if soup.title else "No title found"
        body = soup.body
        html_table_data = ""
        if body:
            for irrelevant in body.find_all(["script", "style", "img", "input"]):
                irrelevant.decompose()
            html_table_data = soup.find("table")

        else:
            print("Warning: No <body> tag found in the page. Check the URL or content.")
        
        html_table_data = soup.find("table")
        return title, html_table_data

    


In [10]:
def build_stock_analysis_prompt(stock_symbol,title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."

    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [23]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = StockWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
         model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [25]:
display_analysis("GOOG")

It seems you didn't provide the actual HTML table data you mentioned for GOOG stock. However, I can guide you on how to analyze stock data, summarize trends, and calculate returns based on historical data.

### Analyzing Stock Data

1. **Understanding Key Terms**:
    - **Open Price**: The price of the stock at the start of the trading week.
    - **Close Price**: The price at the end of the trading week.
    - **High**: The maximum price during the week.
    - **Low**: The minimum price during the week.

2. **Identifying Trends**:
   - **Price Movement**: Look at the 'Open' and 'Close' prices over the weeks. If the close price is consistently higher than the open price, it indicates a bullish trend (the stock is gaining).
   - **Volatility**: Check the differences between high and low prices. Large fluctuations might suggest high volatility, meaning the stock price is changing significantly week to week.

3. **Overall Sentiment**:
   - If there are more weeks with increasing close prices overall compared to those with decreasing close prices, it shows positive sentiment.

### Calculating Total Returns

To calculate the total returns over the period:

1. **Formula**:
   \[
   \text{Total Return (\%)} = \left( \frac{\text{Final Close Price} - \text{Initial Open Price}}{\text{Initial Open Price}} \right) \times 100
   \]

2. **Example Calculation**:
   - Suppose the **Initial Open Price** (first week) is $1000 and the **Final Close Price** (last week) is $1200.
   \[
   \text{Total Return (\%)} = \left( \frac{1200 - 1000}{1000} \right) \times 100 = 20\%
   \]

### Suggestions for Your Analysis
- Look for patterns in the weekly close prices to determine whether the stock has been on an upward or downward trajectory.
- Note any significant jumps or drops; these may correlate with market news or company announcements.

### Summary
To provide a thorough analysis and return calculation, please supply the actual stock data. If you obtain the data in a readable format, I can help interpret that specific information for you.

In [26]:
display_analysis("TSLA")

It seems that there was an issue with the transmission of the data as you mentioned "None" instead of the actual HTML table for TSLA. Without the data provided, I cannot perform the analysis or summarize the trends.

However, I can guide you on how to analyze stock trends and calculate total returns:

### Understanding Stock Trends

1. **Open and Close Prices**:
   - The **open price** is the price of the stock at the start of the trading week.
   - The **close price** is how much the stock was worth at the end of that week.
   - By comparing these two prices over time, you can see if the stock is generally going up (bullish trend) or down (bearish trend).

2. **Highs and Lows**:
   - The **high** indicates the highest price of the stock for that week, while the **low** indicates the lowest price.
   - An increasing high may suggest that the stock is gaining momentum.

3. **Weekly Trends**:
   - Look for patterns over the weeks. Are the closing prices generally moving higher? Is there a consistent pattern of rising or falling prices?

### Calculating Total Returns

To calculate the total returns in percentage:
1. **Find the Initial and Final Prices**:
   - Identify the closing price from the first week (initial price) and the closing price from the last week (final price).

2. **Use the Total Returns Formula**:
   \[
   \text{Total Returns \%} = \left(\frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}}\right) \times 100
   \]
This formula helps quantify the gain or loss over the entire period as a percentage of the initial investment.

### Example Calculation (Hypothetical)

- **Initial Price** (1st week close): $500
- **Final Price** (last week close): $600

Using the formula:
\[
\text{Total Returns \%} = \left(\frac{600 - 500}{500}\right) \times 100 = \left(\frac{100}{500}\right) \times 100 = 20\%
\]

### Summary
To analyze the stock TSLA, you would focus on its opening and closing prices to identify trends, as well as highs and lows for market fluctuations. Once you have the necessary data, you can calculate the total returns over the specified period.

If you manage to receive the data again, I can assist in providing a detailed analysis based on the actual figures.